August 19

- https://claude.ai/chat/f1fa15b1-17b3-4356-9119-ab2cac0b8b84

tutorial told me to wrote this instead. with the secondary argument being an empty array only []

why is this working? as in why is empty array preventing it being called infinitely, but [fireDB, todosRef] not working? fyi fireDB is firestore initialised from config

---

The empty array `[]` as the second argument to `useEffect` is working because it tells React to run the effect only once, when the component mounts. This prevents the effect from running on every render, which could lead to infinite loops or unnecessary database calls.

Here's why:

1. `useEffect(() => {...}, [])`: The empty array means the effect has no dependencies. It runs once after the initial render and never again.

2. `useEffect(() => {...}, [fireDB, todosRef])`: This would run the effect whenever `fireDB` or `todosRef` change. If these are being recreated on every render (which is likely if they're not memoized), it could cause the effect to run repeatedly.

The reason `[fireDB, todosRef]` might be causing issues:

1. Object identity: If `fireDB` or `todosRef` are created in the component body, they'll have a new identity each render, triggering the effect repeatedly.

2. Initialization timing: If `fireDB` is initialized from a config, it might not be fully ready when the effect first runs, causing errors.

3. Circular dependencies: The effect might be updating state that causes a re-render, which then recreates `fireDB` or `todosRef`, triggering the effect again.

To fix this while still properly declaring dependencies:

1. Ensure `fireDB` and `todosRef` are stable (use `useMemo` or initialize outside the component).
2. If they're truly constant, you can define them outside the component entirely.
3. If the Firestore instance is needed only for this effect, consider initializing it inside the effect.

Example of a potential fix:

```javascript
const fireDB = useMemo(() => firebase.firestore(), []);
const todosRef = useMemo(() => fireDB.collection('todos'), [fireDB]);

useEffect(() => {
  let unsubscribe = onSnapshot(todosRef, (snapshot) => {
    // ... rest of your code
  });
  return () => unsubscribe();
}, [todosRef]); // Now todosRef is stable and can be a dependency
```

This approach ensures the effect runs when needed without causing infinite loops.